In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('spamdetection').getOrCreate()

In [3]:
data = spark.read.csv('SMSSpamCollection',inferSchema=True,sep='\t')

In [4]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [5]:
#rename columns
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [6]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [7]:
from pyspark.sql.functions import length

In [8]:
data = data.withColumn('length',length(data['text']))

In [9]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [10]:
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [11]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer

In [12]:
tokenizer = Tokenizer(inputCol='text',outputCol='token_text')

In [13]:
stop_remove = StopWordsRemover(inputCol='token_text',outputCol='stop_token')

In [14]:
count_vec = CountVectorizer(inputCol='stop_token',outputCol='c_vec')

In [16]:
idf = IDF(inputCol='c_vec',outputCol='tf_idf')

In [18]:
ham_spam_to_numeric = StringIndexer(inputCol='class',outputCol='label')

In [19]:
from pyspark.ml.feature import VectorAssembler

In [20]:
clean_up =  VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [21]:
from pyspark.ml.classification import NaiveBayes

In [22]:
nb = NaiveBayes()

In [23]:
from pyspark.ml import Pipeline

In [25]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric,tokenizer,stop_remove,count_vec,idf,clean_up])

In [27]:
cleaner = data_prep_pipe.fit(data)

In [28]:
cleaned_data = cleaner.transform(data)

In [29]:
cleaned_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [30]:
cleaned_data =cleaned_data.select('label','features')

In [31]:
cleaned_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [32]:
training,test =cleaned_data.randomSplit([0.7,0.3])

In [33]:
spam_detector = nb.fit(training)

In [34]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [35]:
test_results = spam_detector.transform(test)

In [36]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,13,...|[-603.79185913615...|[1.0,2.1655477448...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-577.47157503807...|[0.99999999991762...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-705.08444200546...|[1.0,1.7247273903...|       0.0|
|  0.0|(13424,[0,1,17,19...|[-821.10152545621...|[1.0,2.0548203034...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-972.95412064076...|[1.0,2.1219871829...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-767.43697111032...|[1.0,1.0152396709...|       0.0|
|  0.0|(13424,[0,1,27,88...|[-1525.2833033996...|[0.99796148129726...|       0.0|
|  0.0|(13424,[0,1,43,69...|[-631.89132513160...|[0.24605942842980...|       1.0|
|  0.0|(13424,[0,1,72,10...|[-697.49975398873...|[1.0,9.3160831402...|       0.0|
|  0.0|(13424,[0

In [37]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [38]:
acc_eval = MulticlassClassificationEvaluator()

In [40]:
acc = acc_eval.evaluate(test_results)

In [41]:
print(acc)

0.9154720534897847
